In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import regexp_extract, when, trim, col

# Initialisation de la session Spark

In [2]:
spark = SparkSession.builder \
    .appName("DataMergingWithHive") \
    .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

# Utilisation de la base de données Hive, et charge les données des tables traitées

In [3]:
spark.sql("USE concessionnaire")

clients_df = spark.sql("SELECT * FROM client_processed")
immatriculations_df = spark.sql("SELECT * FROM immatriculations_processed")

# Fusionner les tables sur la colonne 'Immatriculation'

In [4]:
merged_df = clients_df.join(immatriculations_df, "Immatriculation", "inner")


In [5]:
merged_df.show(n=10, truncate=False)

+---------------+---+-----+----+-------------------+---------------+---------------+--------+---------+-----------+--------+--------+-------+--------+-------+------+
|immatriculation|age|sexe |taux|situationfamilliale|nbenfantacharge|deuxiemevoiture|marque  |puissance|longueur   |nbplaces|nbportes|couleur|occasion|prix   |modele|
+---------------+---+-----+----+-------------------+---------------+---------------+--------+---------+-----------+--------+--------+-------+--------+-------+------+
|0 UI 76        |61 |Homme|419 |En Couple          |4              |true           |Mercedes|306      |très longue|5       |5       |rouge  |true    |70910.0|S500  |
|10 IM 37       |20 |Homme|1245|En Couple          |2              |false          |Jaguar  |197      |longue     |5       |5       |bleu   |false   |37100.0|X-Type|
|10 LB 49       |21 |Homme|502 |En Couple          |2              |false          |Volvo   |272      |très longue|5       |5       |rouge  |false   |50500.0|S80   |
|10 

# Enregistrer le DataFrame fusionné sous forme de table Hive

In [6]:
merged_df.createOrReplaceTempView("merged_view")
spark.sql("DROP TABLE IF EXISTS client_immatriculation_merged")
spark.sql("CREATE TABLE client_immatriculation_merged AS SELECT * FROM merged_view")

DataFrame[]